# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Julian Endres" <br>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########

import re
import pandas as pd

######## Plotting ###########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
#import plotly.offline as pltly
#import plotly.io as pio

In [ ]:
# specify what to import (in path ~/.windnode_abw/)
run_timestamp = '2020-07-24_104145_1month'
scenario = 'NEP2035'

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=[scenario])

# TODO
- [ ] parallelize analysis
- [ ] save / pickle results

### useful params

In [ ]:
MUN_NAMES = regions_scns[scenario].muns.gen.to_dict()
bar_colors = px.colors.sequential.GnBu_r
UNITS = {"relative": "%", "hours": "h",}

# 1 Demand and Generation (Input Data)

## 1.1 Installed Electrical Capacities

In [ ]:
df_data = results_scns[scenario]['parameters']['Installed capacity electricity supply']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(3,3, figsize=(12,10))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='MW')
    
fig.suptitle('Installed el. Generation Capacity',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.2 Electrical Demand

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
df_data = df_data.drop(columns='export')
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = df_data / 1e3

fig, axes = plt.subplots(1,3, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')

fig.suptitle('Electrical Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.3 Thermal Demand

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Wärmenachfrage'].sum(level=2)
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = pd.DataFrame([df_data.iloc[:,:2].sum(axis=1).rename('Households'), df_data.iloc[:,-1]]).T
df_data = df_data / 1e3

fig, axes = plt.subplots(1,2, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.items()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')
    
fig.suptitle('Thermal Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

# 2 Area required by RES

## 2.1 Absolute Area

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Area required']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(2,2, figsize=(12,6))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='ha')
    
fig.suptitle('Required Area',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 2.2 Relative Area

The following figure shows the total relative area required by the RES in ABW for the current scenario compared to the available areas for different land use scenarios, per RES technology.

Notes:
- The current scenarios of wind turbines and ground-mounted PV is marked with "(current)"
- Technology-specific naming conventions of land use scenarios
  - Wind: Distance to settlements (500m/1000m), use of forests (with/without), percentage of available area due to restrictions resulting from case-by-case decisions
  - PV ground: Restrictions that apply (hard: H / hard+weak: HS), percentage of total available agricultural area

In [ ]:
df_data = results_scns[scenario]['highlevel_results']

fig = go.Figure()

# PV rooftop
mask = [i for i in df_data.index if 'PV rooftop' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV rooftop',
           marker_color=bar_colors[5]))

# PV Ground
mask = [i for i in df_data.index if 'PV ground' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV Ground',
           marker_color=bar_colors[4]))#, visible='legendonly'))

# Wind
mask = [i for i in df_data.index if 'rel. wind' in  i[0]]
data = df_data.loc[mask]
index=  data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='Wind',
          marker_color=bar_colors[3]))#, visible='legendonly'))

fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)

fig.show()

# 3 Electrical Autarky

Autarky describes how much of the demanded energy can be supplied from sources within the region/municipality with regard to energy balance (annual sum) or simultaneity (number of hours).

- [ ] histogramm timeseries
- [ ] boxplots timeseries (prozentual Energiebilanz)

## 3.1 Supply/Demand

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.sort_values(axis=0, by='relative')

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.17, column_widths=[0.8, 0.2],
                    specs=[[{"secondary_y": True}, {"secondary_y": True}]])

# === municipalities ===
# supply per municipality
data = df_data['supply'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Supply',
          marker_color=bar_colors[1],),
    row=1, col=1, secondary_y=False,)

# demand per municipality
data = df_data['demand'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Demand',
           marker_color=bar_colors[2],),
    row=1, col=1, secondary_y=False,)

# relative autarky per municipality
data = df_data['relative'] * 1e2

fig.add_trace(
    go.Bar(x=data.index, y=data.values,
           orientation='v',
           name='Relative',
           marker_color=bar_colors[3],
           visible='legendonly',
          opacity=0.7,),
    row=1, col=1, secondary_y=True,)

# === ABW ===
df_data = results_scns[scenario]['results_t']['Autarky']

fig.add_trace(
    go.Bar(x=['supply'], y=[df_data['supply']/ 1e3],
           orientation='v',
           name='ABW Supply',
           marker_color=bar_colors[1]), 
    row=1, col=2, secondary_y=False,)

fig.add_trace(
    go.Bar(x=['demand'], y=[df_data['demand'] / 1e3],
           orientation='v',
           name='ABW Demand',
           marker_color=bar_colors[2],),
    row=1, col=2, secondary_y=False,)

fig.add_trace(
    go.Bar(x=[df_data.name], y=[df_data['relative']* 1e2],
           orientation='v',
           name='ABW relative',
           marker_color=bar_colors[3],
           visible='legendonly',),
    row=1, col=2, secondary_y=True,)

# === Layout ===
fig.update_layout(title_text = 'Electrical Autarky per Municipality (Energy Balance) and total region',
                    autosize=True,
                hovermode="x unified",
                  legend=dict(orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                 )
fig.update_yaxes(title_text="absolute in GWh", row=1, col=1, anchor="x", secondary_y=False)
fig.update_yaxes(title_text="relative in %", row=1, col=1, anchor="x", secondary_y=True)
fig.update_yaxes(title_text="absolute in GWh", row=1, col=2, anchor="x2", secondary_y=False)
fig.update_yaxes(title_text="relative in %", row=1, col=2, anchor="x2", secondary_y=True)
fig.show()

## 3.2 Relative Autarky

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky']['relative']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.mul(100)
df_data = df_data.sort_values()

data=df_data.round()

limit=120

# split data
data_left = data[data < limit]
data_right = data[data >= limit]

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.2)

fig.add_trace(
    go.Bar(y=data.index, x=data_left.values,
           orientation='h', name=f'< {limit} %',
          marker_color=bar_colors[1]),
    row=1, col=1)

fig.add_trace(
    go.Bar(y=data.index, x=data_right.values,
           orientation='h', name=f'> {limit} %',
          marker_color=bar_colors[1]),
    row=1, col=2)

fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)
fig.update_xaxes(title='%', anchor='x', row=1, col=1)
fig.update_xaxes(title='%', anchor='x2', row=1, col=2)

fig.update_layout(title_text='Grouped Electrical Autarky per Municipality (Energy Balance)',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)
fig.show()

## 3.3 relative time

- [ ] Summe über alle Gemeinde als TS zusätzlich?
- [ ] relativer Zeitliche Anteil Autarky 10% aller stunden?? was habe ich hiermit gemeint?

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky'][['hours']]
max_hours  = len(results_scns[scenario]['flows_txaxt']['Autarky'].index.get_level_values(level=0).unique())
df_data = df_data['hours'] / max_hours * 1e2

df_data = df_data.sort_values(ascending=False)
df_data = df_data.rename(index=MUN_NAMES)

fig = go.Figure()
fig.add_trace(
    go.Bar(x=df_data.index,
           y=df_data.values,
           orientation='v',
           marker_color=bar_colors[1]))

fig.update_yaxes(title='%')
fig.update_layout(
    title='Percentage of Autark Hours', 
    showlegend=False)
fig.show()

## 3.4 Autarky Geoplots

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Autarky'].loc[:,['hours', 'relative']]
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(1,2, figsize=(12,5))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit=UNITS[key])
    
fig.suptitle('Autark Hours and Relative Autarky Balance',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 3.5 violinplot

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Autarky']['relative'].unstack()

df_data.columns = [int(i) for i in df_data.columns]
df_data = df_data.rename(columns=MUN_NAMES)

limit = 2

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.25,
                    row_heights=[0.8, 0.2],
                    specs=[[{}, {}], [{"colspan": 2}, None]],)

for ags, data in df_data.iteritems():
    
    if data.describe().loc['mean'] > limit:

        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=False),
                              
                      row=1, col=1)
    else:
        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=False),
                      row=1, col=2)
        
    fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=bar_colors[1],
                               showlegend=True,
                           visible='legendonly'),
                              
                      row=2, col=1)
    
fig.update_xaxes(title_text="%", side='bottom', row=2, col=1)
fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)

fig.update_layout(
    title='Relative Electrical Autarky per Municipality (Energy Balance)')
fig.show()

# 4 Energy Mix

INFO:
- export == ins nationale Netz
- ABW_... == intra_regional

TODO:

## 4.1 Balance

In [ ]:
supply = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].sum(level=1)
abw_import = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['import'].sum(level=1)
abw_import = abw_import.rename('ABW-import')
supply = supply.join(abw_import)


demand = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
abw_export = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['export'].sum(level=1)
abw_export = abw_export.rename('ABW-export')
demand = demand.join(abw_export)
el_heating = results_scns[scenario]['flows_txaxt']['Stromnachfrage Wärme'].sum(level=2).sum(axis=1)
demand = demand.join(el_heating.rename('el_heating'))
    
plot_snd_total(regions_scns[scenario], supply , demand)

## 4.2 Time

In [ ]:
plot_timeseries(results_scns[scenario], kind='el')#, ags='15091160')

# 5 Emissions

TODO
- [ ] große Werte besonders schöne Farben
- [ ] Notiz zur Zuordnung von KWK zu el etc

In [ ]:
df_data_left = results_scns[scenario]['results_t']['CO2 emissions th. total'].rename('th').to_frame()
df_data_right = results_scns[scenario]['results_t']['CO2 emissions el. total'].rename('el').to_frame()

# drop nans & zeros
df_data_left = df_data_left[df_data_left!=0].dropna()
df_data_right = df_data_right[df_data_right!=0].dropna()

df_data = df_data_left.join(df_data_right, how='outer')
df_data = df_data.fillna(0).T
df_data = df_data.sort_values(by=list(df_data.index), axis=1, ascending=True)
#df_rel = df_data.T / df_data.sum(axis=1).values

fig = px.bar(df_data, orientation='h',
             color_discrete_sequence=bar_colors,
             text=df_data.sum(axis=1).to_list()
            )


fig.update_layout(barmode='stack',
                  autosize=True,
                  title='CO2 Emissions',
                  legend={'traceorder':'reversed'},
                 uniformtext_mode='hide')
fig.update_traces(hovertemplate='CO2: %{x:.1f} t <br>Type: %{y}<br>Total: %{text:.1f} t') # 
fig.update_xaxes(title_text='t CO2')
fig.update_yaxes(title_text='')
fig.show()

# 6 Costs

## 6.1 LCOE and LCOH

<div class="alert alert-block alert-info">
<b>Notes on LCOE calculation</b>

- Total LCOE calculate as $LCOE=\frac{expenses_{el.total}}{demand_{el.,total}}$, likewise total LCOH calculate as $LCOH=\frac{expenses_{th.,total}}{demand_{th.,total}}$
- Total expenses $expenses_{el.total}$ are annual expenses. Investment costs are discounted to one year using equivalent periodic costs
- The plot below shows fractions of these LCOE that are calculated as $LCOE_{technology}=\frac{expenses_{el.,technology}}{demand_{el.,total}}$ representation the share of each technology at total cost of one MWh
</div>

In [ ]:
values = ['LCOE','LCOH']

df = pd.DataFrame([results_scns[scenario]['results_t'][i] for i in values], index=values)
df = df.rename(columns=PRINT_NAMES)
df = df.sort_values(by=values, axis=1, ascending=True)

fig = px.bar(df, orientation='h',
             title='LCOE and LCOH',
            color_discrete_sequence=bar_colors,
                text=df.sum(axis=1).to_list())

fig.update_layout(barmode='stack', legend={'traceorder':'reversed'},
                  uniformtext_mode='hide',
                 )
fig.update_traces(hovertemplate='Share: %{x:.1f}€ <br>Type: %{y}<br>Total: %{text:.1f}€'  ) # 
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

# 7 Power Grid
## 7.1 Maximum Line Loading

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].max(level=1) * 100
df_data = df_data.sort_index(ascending=False)
df_data = pd.DataFrame().from_dict({'line loading': df_data,'free capacity': 60-df_data})

index = [re.split(r'(\d+)', s) for s in df_data.index]
df_data.index = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in index]

fig = go.Figure() 
for i, (key, df) in enumerate(df_data.items()):

    fig.add_bar(y=df.index,
                x=df.values,
                orientation='h',
                name=key,
                marker_color=bar_colors[4*i+2],
                hovertemplate='%{x:.2f}%',
                showlegend=False)

fig.update_layout(barmode="relative",
    title='Maximum Line Loading',
    yaxis_tickfont_size=12,
    xaxis=dict(
        title='Loading in %',
        titlefont_size=16,
        tickfont_size=12,
    ))

fig.update_yaxes(type='category')
fig.update_layout(width=800, hovermode="y unified")
fig.show()


## 7.2 Line Loading Distribution

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading per bus']
index = df_data.index.get_level_values(level=0)

fig = go.Figure()
# limit = 2
# fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.25)
# for ags, data in df_data.iteritems():
#     if data.describe().loc['mean'] > limit:

fig.add_trace(go.Box(
    y=df_data.values * 1e2,
    x=index,
    name='Increase',
    marker_color=bar_color[5],
    boxpoints=False,))

fig.update_layout(
    title='Line Loading',
    yaxis_title='in %',
    boxmode='group')

fig.update_xaxes(type='category', tickangle=45)
fig.show()

## 7.3 Mean Line Loading

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].mean(level=['ags_from','ags_to'])
df_data = df_data.sort_index(ascending=False)

x = df_data.index.get_level_values(level='ags_from')
x = [re.split(r'(\d+)', s) for s in x]
x = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in x]

y = df_data.index.get_level_values(level='ags_to')
y = [re.split(r'(\d+)', s) for s in y]
y = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in y]

fig = go.Figure(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    hovertemplate='-> %{y}<br>'+
    'Mean Loading %{marker.size:.1f}%'+
    '<extra></extra>',
    marker=dict(
        size=df_data.values * 1e2 ,
        sizemode='diameter',
        sizeref=2*df_data.max(),
        sizemin=4,
        #showscale=True,
        color=-df_data.values,
        colorscale=bar_color,),
))
fig.update_layout(title={
        'text': 'Mean Line Loading in %',
         'x':0.5},
                  hovermode="x unified")

fig.update_xaxes(type='category', tickangle=45)
fig.update_yaxes(type='category')
fig.show()

## 7.4 Mean Line Loading Heatmap

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].mean(level=['ags_from','ags_to'])
df_data = df_data.sort_index(ascending=False)

hover_text = [ f'Mean loading:{int(i *1e2)}%<br>' for i in df_data.values]

x = df_data.index.get_level_values(level='ags_from')
x = [re.split(r'(\d+)', s) for s in x]
x = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in x]

y = df_data.index.get_level_values(level='ags_to')
y = [re.split(r'(\d+)', s) for s in y]
y = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in y]


fig = go.Figure(go.Heatmap(
    x=x,
    y=y,
    z=df_data.values * 1e2,
    colorscale=bar_color,
    colorbar=dict(title='%'),
    hovertext=hover_text,
    showscale=True,
))
fig.update_layout(title='Line Loading',
                  hovermode="x unified")
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

# 8 Flexibility

- radar chart https://plotly.com/python/radar-chart/

## 8.1 Heatstorage

<div class="alert alert-block alert-info">
<b>Notes on Heatstorage Charts</b>
  
- the relative values on the secondary axis only acount for the municipalities!
    
- To show the total cycles, only the discharge values are chosen as the difference are the storage losses and almost negligibly  

</div>
    

In [ ]:
def get_timesteps(region):
    timestamps = pd.date_range(start=region._cfg['date_from'],
                               end=region._cfg['date_to'],
                               freq=region._cfg['freq'])
    steps = len(timestamps)
    return steps

In [ ]:
discharge = results_scns[scenario]['parameters']['Discharge power heat storage']
discharge = discharge.rename(columns={'stor_th_large':'cen','stor_th_small':'dec'})

capacity = results_scns[scenario]['parameters']['Installed capacity heat storage']

nominal_power = results_scns[scenario]['results_axlxt']['Wärmespeicher nach Gemeinde']
nominal_power = nominal_power.discharge.unstack().fillna(0).T
nominal_power.index = nominal_power.index.astype(int)

# total 
total_cycle = (discharge / capacity.values).fillna(0)

# relative
steps = get_timesteps(regions_scns[scenario])
max_cycle = (1/2 * steps *  nominal_power) / capacity.values
max_cycle = max_cycle.fillna(0)

full_load_cycles = total_cycle / max_cycle
full_load_cycles = full_load_cycles.fillna(0)

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    horizontal_spacing=0.05,
                    vertical_spacing=0.4,
                    column_widths=[0.2, 0.8],
                    subplot_titles=("Central", "Decentral"),
                   specs=[[{"secondary_y": True}, {"secondary_y": True}]])

for i, (key, df) in enumerate(flex_ratios_heat.groupby(level=0)):
    
    df_data = df.loc[(key, slice(None))]
    df_data.index = df_data.index.astype(int)
    df_data = df_data.rename(index=MUN_NAMES)
    
    # --- total ---
    fig.add_trace(
        go.Bar(y=df_data['total_cycles'], x=df_data.index,
               orientation='v',
               name='municipalities',
               legendgroup="muns",
               marker_color='rgb(107,174,214)',
              showlegend=bool(i),
              hovertemplate = 'total cycles: %{y:.2f} times<extra></extra>',),
        row=1, col=i+1, secondary_y=False)

    # --- relative ---
    fig.add_trace(
        go.Bar(y=df_data['relative_cycles'] *1e2, x=df_data.index,
               orientation='v',
               name='relative',
               legendgroup="muns",
               marker_color='rgb(107,174,214)',
               opacity=0.5,
               showlegend=False,
              hovertemplate = 'relative cycles: %{y:.2f} %<extra></extra>'),
        row=1, col=i+1, secondary_y=True)

    # --- ABW ---
    fig.add_trace(
        go.Bar(y=[df_data['total_cycles'].sum()], x=['ABW'],
               orientation='v',
               name='ABW',
               legendgroup="ABW",
               marker_color='rgb(8,81,156)',
               showlegend=bool(i),
               visible='legendonly',
              hovertemplate = 'total cycles: %{y:.2f} times<extra></extra>'),
        row=1, col=i+1, secondary_y=False)
    
    fig.add_trace(
        go.Bar(y=[df_data['relative_cycles'].mean() *1e2], x=['ABW'],
               orientation='v',
               name='Mean',
               legendgroup="ABW",
               marker_color='rgb(9,56,125)',
               showlegend=False,
               visible='legendonly',
              hovertemplate = 'mean relative cycles: %{y:.2f} %<extra></extra>'),
        row=1, col=i+1, secondary_y=True)  
    

# === Layout ===

fig.update_layout(title='Heat storage charge-cycles',
                    autosize=True,
                   hovermode="x unified",
                  legend=dict(orientation="h",
                                yanchor="bottom",
                                y=1.05,
                                xanchor="right",
                                x=1))
fig.update_yaxes(title_text="total", row=1, col=1, anchor="x", secondary_y=False)
fig.update_yaxes(title_text="relative", row=1, col=2, anchor="x2", secondary_y=True)
fig.update_xaxes(type='category', tickangle=45)
fig.show()

### TODO
- gesamtregion? als scalare Tabelle. Wie genau soll das gehen?

    - total cycle decentral, central sum()
    - relativ auf theoretisch mögliche cycles: 1/2  * 8760 / (nom_cap / nom_discharge_power)
    
- Tabelle Ausdrucken Plotly tables?

### total ABW

## 8.2 Batterystorage

## TODO
- add large/small
- auf Kapazität beziehen: total cycles
- nominale Entladeleistung: VLS
- mit NEP testen. was passiert wenn kein small vorhanden?!

Fragen:
- warum discharge small/large
- capacity nicht?
Problem:
- 

## break

In [ ]:
discharge = results_scns[scenario]['results_axlxt']['Batteriespeicher nach Gemeinde']['discharge']
capacity = results_scns[scenario]['parameters']['Installierte Kapazität Großbatterien']

flex_ratios_bat = pd.DataFrame(index=discharge.index)

total_cycle_small = discharge.small / capacity.loc[discharge.small.index.astype(int), 'capacity'].values
total_cycle_large = discharge.large / capacity.loc[discharge.large.index.astype(int), 'capacity'].values

flh_small = discharge.small / capacity.loc[discharge.small.index.astype(int), 'power_discharge'].values 
flh_large = discharge.large / capacity.loc[discharge.large.index.astype(int), 'power_discharge'].values 


flex_ratios_bat.loc[('small', slice(None)), 'total_cycles'] = total_cycle_small.values
flex_ratios_bat.loc[('large', slice(None)), 'total_cycles'] = total_cycle_large.values
flex_ratios_bat.loc[('small', slice(None)), 'FLC'] = flh_small.values
flex_ratios_bat.loc[('large', slice(None)), 'FLC'] = flh_large.values

# relative
steps = get_timesteps(regions_scns[scenario])
max_cycle_small = (1/2 * steps *  capacity.loc[discharge.small.index.astype(int), 'power_discharge']) \
/ capacity.loc[discharge.small.index.astype(int), 'capacity']

max_cycle_large = (1/2 * steps *  capacity.loc[discharge.large.index.astype(int), 'power_discharge']) \
/ capacity.loc[discharge.large.index.astype(int), 'capacity']
                                                                
flex_ratios_bat.loc[('small', slice(None)), 'relative_cycles'] = total_cycle_small.values / max_cycle_small.values
flex_ratios_bat.loc[('large', slice(None)), 'relative_cycles'] = total_cycle_large.values / max_cycle_large.values


### 8.2.1 Cycles

In [ ]:
fig = make_subplots(rows=1, cols=2,
                     horizontal_spacing=0.1,
                    vertical_spacing=0.4,
                    column_widths=[0.5, 0.5],
                    subplot_titles=("small", "large"),
                   specs=[[{"secondary_y": True}, {"secondary_y": True}]])


for i, (key, df) in enumerate(flex_ratios_bat.groupby(level=0)):
    
    df_data = df.loc[(key, slice(None))]
    df_data = df_data.loc[(df_data != 0).any(axis=1)]
    df_data.index = df_data.index.astype(int)
    df_data = df_data.rename(index=MUN_NAMES)
    
    # --- total ---
    fig.add_trace(
        go.Bar(y=df_data['total_cycles'], x=df_data.index,
               orientation='v',
               name='total',
               marker_color='rgb(107,174,214)',
               showlegend=bool(i),
               legendgroup="total",
              hovertemplate = 'Total Cycles: %{y:.2f} times<extra></extra>'),
        row=1, col=i+1, secondary_y=False)
    
    # --- FLH ---
    fig.add_trace(
        go.Bar(y=df_data['FLC'], x=df_data.index,
               orientation='v',
               name='FLC',
               marker_color='rgb(9,56,125)',
               #opacity=0.5,
               showlegend=bool(i),
               legendgroup="FLH",
               #visible='legendonly',
              hovertemplate = 'Full Load Cycles: %{y:.2f} times<extra></extra>'),
        row=1, col=i+1, secondary_y=False)
    
    # --- relative ---
    fig.add_trace(
        go.Bar(y=df_data['relative_cycles'] * 1e2, x=df_data.index,
               orientation='v',
               name='relative',
               marker_color='rgb(107,174,214)',
               opacity=0.5,
               showlegend=bool(i),
               legendgroup="relative",
              hovertemplate = 'Relative Cycles: %{y:.2f} %<extra></extra>'),
        row=1, col=i+1, secondary_y=True)

fig.update_layout(title='Battery storage charge cycles ',
                  legend={'traceorder':'grouped'},
                 hovermode="x unified")

fig.update_yaxes(title_text="total cycles", row=1, col=1, anchor="x", secondary_y=False)
fig.update_yaxes(title_text="relative capacity in %?", row=1, col=2, anchor="x2", secondary_y=True)
fig.update_xaxes(type='category', tickangle=45)
fig.show()

### 8.2.2 Timeseries

In [ ]:
# timeseries
df_data = results_scns[scenario]['flows_txaxt']['Batteriespeicher'].sum(level=0)
# only get nonzero values
df_data = df_data.loc[(df_data != 0).any(axis=1)]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df_data.index, y=df_data['discharge'],
                    mode='markers',
                    name='discharge'))
fig.add_trace(go.Scatter(x=df_data.index, y=df_data['charge'],
                     mode='markers',
                     name='charge'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
#                     mode='lines',
#                     name='lines'))
fig.update_layout(title='Battery Charge/Discharge')
fig.update_yaxes(title_text='?')
fig.show()

- in relation zu anderen Werten
- scatter: Wind+PV zu Batt-charge
- mit Jahr ausprobieren

## 8.3 DSM

- bubblemap
    - größe activation
    - x RE, Line Loadings, Battery Storage
    - y auch

- Bezug auf hh-demand-mit-dsm beziehen?
    - auf total
    - auf hh-demand
- absolute dsm activation
- pro AGS

- outliers durch linie erstzen

### 8.3.1 Activation

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['DSM activation']

x = df_data.index.get_level_values(level=1)
x = pd.Series(x.astype(int).values).map(MUN_NAMES).values

fig = go.Figure()

fig.add_trace(go.Box(
    y=df_data['Demand increase'].values,
    x=x,
    name='Increase',
    marker_color='#3D9970',
    boxpoints=False,

))
fig.add_trace(go.Box(
    y=df_data['Demand decrease'].values,
    x=x,
    name='Decrease',
    marker_color='#FF4136',
    boxpoints=False,

))

fig.update_layout(
    yaxis_title='DSM activation in MWh',
    boxmode='group'
)

fig.update_xaxes(type='category', tickangle=45)a
fig.show()


### 8.3.2 DSM Total Sum

- barplot Summe/hh_demand

### 8.3.3 DSM Timeseries

In [ ]:
ags = '15091060'
ags = str(ags)

df_data = results_scns[scenario]['flows_txaxt']['DSM activation']


if ags== 'ABW':
    df_data = df_data.sum(level='timestamp')
else:
    df_data = df_data.loc[(slice(None),ags),:].sum(level=0)


fig = go.Figure()

fig.add_trace(go.Scatter(x=df_data.index,
                             y=df_data['Demand increase'].values,
                             name='Demand increase',
                         mode='lines',
                             #fill='tonexty',
                             #mode='none',
                             #fillcolor=COLORS[tech],
                            #stackgroup='one'
                        ))


fig.add_trace(go.Scatter(x=df_data.index,
                             y=df_data['Demand decrease'],
                             name='Demand decrease',
                         mode='lines',
                             #fill='tonexty',
                             #mode='none',
                             #fillcolor=COLORS[tech],
                            #stackgroup='two'
                        ))


fig.update_xaxes(
    title='Zoom',
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=14, label="2w", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=3, label="3d", step="day", stepmode="backward"),
            #dict(step="all")
        ])
    )
)

fig.update_layout(
    title='Demand Side Management of %s'% ags,
    height = 700,
    #xaxis={'categoryorder':'category ascending'},
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='MW',
        titlefont_size=16,
        tickfont_size=14),
    autosize=True,
    )
fig.show()


- hh demand zusätzlich

# 9 Energy Exchange

## 9.1 Supply/Demand

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Intra-regional exchange']
df_data = df_data.rename(index=MUN_NAMES)

fig = go.Figure()
# fig.add_trace(go.Bar(x=df_data.index, y=df_data['export'].values ,
#                 base=df_data['export'].values * -1,
#                 marker_color='crimson',
#                 name='export'))

fig.add_trace(go.Bar(x=df_data.index, y=df_data['export'].values, 
                base=0,
                marker_color='crimson',
                name='export'
                ))


fig.add_trace(go.Bar(x=df_data.index, y=df_data['import'].values, 
                base=0,
                marker_color='lightslategrey',
                name='import'
                ))
fig.update_xaxes(type='category', tickangle=45)
fig.show()

## 9.2 AGS Exchange Bubbleplot

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']

# swap index of negative values
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)

df_data = df_data['out']
df_data = df_data.sort_index()

hover_text = [f' From: {MUN_NAMES[int(ags_from)]} <br> \
To:{MUN_NAMES[int(ags_to)]} <br> \
Energy: {int(value)} MWh' for (ags_from, ags_to), value in df_data.items()]

x = df_data.index.get_level_values(level='ags_from')
x = pd.Series(x.astype(int).values).map(MUN_NAMES).values

y = df_data.index.get_level_values(level='ags_to')
y = pd.Series(y.astype(int).values).map(MUN_NAMES).values

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    text= hover_text,
    showlegend=False, 
    marker=dict(
        size=df_data.values ,
        sizemode='diameter',
        sizeref=2.*df_data.max()/(13**2),
        sizemin=4,
        showscale=True,
        color=df_data.values,
        colorscale='viridis'

    ),
    
))

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    #text=False,# hover_text,
    fillcolor='red',
    showlegend=False,

))


fig.update_layout(title='Intra Regional Energy Exchange')#, xaxis_showgrid=True, yaxis_showgrid=True)
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

## 9.3 Heatmap

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']

# swap index of negative values
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)

df_data = df_data['out']
df_data = df_data.sort_index()


hover_text = [f' From: {MUN_NAMES[int(ags_from)]} <br> \
To:{MUN_NAMES[int(ags_to)]} <br> \
Energy: {int(value)} kWh' for (ags_from, ags_to), value in df_data.items()]

x = df_data.index.get_level_values(level='ags_from')
x = pd.Series(x.astype(int).values).map(MUN_NAMES).values

y = df_data.index.get_level_values(level='ags_to')
y = pd.Series(y.astype(int).values).map(MUN_NAMES).values

fig = go.Figure(go.Heatmap(
    x=x,
    y=y,
    z=df_data.values,
    colorscale='Viridis',
    hoverongaps = True
#     mode='markers',
#     text= hover_text,
#     marker=dict(
#         size=df_data.values ,
#         sizemode='diameter',
#          sizeref=2.*df_data.max()/12**2,
#         sizemin=4
#     )
))
fig.update_layout(title='Intra Regional Energy Exchange')
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

## 9.4 Energy Flows

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)


converter = dict(zip(MUN_NAMES.keys(), range(20)))

source = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_from')]
target = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_to')]

In [ ]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = [MUN_NAMES[i] for i in list(converter)],
#       customdata = ["Long name A1", "Long name A2", "Long name A3", "Long name B1",
#                     "Long name B2", "Long name B3"],
        
#       hovertemplate='Node %{customdata} has total value %{value}<extra></extra>',  
      color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = target,
      value =df_data['out'].values,
#         color = ['green', 'yellow', 'black', 'red', 'violet', 'white'],
#       customdata = ["q","r","s","t","u","v"],
        
#       hovertemplate='Link from node %{source.customdata}<br />'+
#         'to node%{target.customdata}<br />has value %{value}'+
#         '<br />and data %{customdata}<extra></extra>',  
  ))])

fig.update_layout(title_text="Energy Flows", font_size=12)
fig.show()

## 9.5 Chord diagram

In [ ]:
import pandas as pd
import holoviews as hv
from holoviews import opts, dim

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)

converter = dict(zip(MUN_NAMES.keys(), range(20)))

source = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_from')]
target = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_to')]
chord_links = pd.DataFrame()
chord_links['source'] = source
chord_links['target'] = target
chord_links['value'] = df_data['out'].values

#names = pd.Series([MUN_NAMES[i] for i in list(converter)]).rename('name')
names = pd.Series(list(MUN_NAMES.values())).rename('name')
chord_nodes = hv.Dataset( names, 'index')

In [ ]:
chord = hv.Chord((chord_links, chord_nodes)).select(value=(20, None))
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), label_index='names',
               #labels='name', # somehow doesnt work
               node_color=dim('index').str()))